## LASSO

Least Absolute Shrinkage Operator 

(Blurb about LASSO)

### Importing Packages

In [12]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt



<IPython.core.display.Javascript object>

### Importing Data/Data Cleaning

In [2]:
df = pd.read_csv("Data/data.csv")
df

,score,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q34,Q35,Q36,Q37,Q38,Q39,Q40,elapse,gender,age
0,18,2,2,2,2,1,2,1,2,2,...,1,1,2,2,2,1,2,211,1,50
1,6,2,2,2,1,2,2,1,2,1,...,2,1,2,2,2,2,1,149,1,40
2,27,1,2,2,1,2,1,2,1,2,...,1,2,1,1,2,1,2,168,1,28
3,29,1,1,2,2,2,1,2,1,1,...,1,2,1,2,2,1,1,230,1,37
4,6,1,2,1,1,1,2,1,2,1,...,2,1,2,2,2,0,1,389,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,2,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,243,1,24
11239,10,2,2,1,1,1,2,1,1,1,...,2,1,2,1,2,2,1,197,1,39
11240,6,1,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,447,2,33
11241,12,2,2,1,1,1,1,1,1,1,...,1,2,2,2,2,2,1,167,1,24


<IPython.core.display.Javascript object>

In [3]:
# Renaming columns
# Will delete after df is saved in EDA
df = df.set_axis(
    [
        "score",
        "Influence",
        "Modesty",
        "Daring",
        "Confidence",
        "Ruling",
        "Responsibility",
        "IntrovExtro",
        "Success",
        "Humble",
        "Leadership",
        "Assertive",
        "Authority",
        "Manipulate",
        "Respect",
        "ShowOffBody",
        "AnalyzePeople",
        "Decision Responsibility",
        "HappyvMeaning",
        "BodyConfidence",
        "ShowOff",
        "KnowWhattoDo",
        "Dependence",
        "Storytelling",
        "Expectations",
        "Satisfaction",
        "Compliments",
        "Power",
        "Trendy",
        "Mirror",
        "CenterofAttention",
        "Freedom",
        "Authority",
        "LeaderPreference",
        "Confidence in Success",
        "MakeBelieve",
        "Leadership Origin",
        "Biography",
        "Public Opinion",
        "Capability",
        "Extraordinary",
        "elapse",
        "gender",
        "age",
    ],
    axis="columns",
    inplace=False,
)

<IPython.core.display.Javascript object>

In [4]:
# Subtract 1 from Question Columns
df[
    [
        "Influence",
        "Modesty",
        "Daring",
        "Confidence",
        "Ruling",
        "Responsibility",
        "IntrovExtro",
        "Success",
        "Humble",
        "Leadership",
        "Assertive",
        "Authority",
        "Manipulate",
        "Respect",
        "ShowOffBody",
        "AnalyzePeople",
        "Decision Responsibility",
        "HappyvMeaning",
        "BodyConfidence",
        "ShowOff",
        "KnowWhattoDo",
        "Dependence",
        "Storytelling",
        "Expectations",
        "Satisfaction",
        "Compliments",
        "Power",
        "Trendy",
        "Mirror",
        "CenterofAttention",
        "Freedom",
        "Authority",
        "LeaderPreference",
        "Confidence in Success",
        "MakeBelieve",
        "Leadership Origin",
        "Biography",
        "Public Opinion",
        "Capability",
        "Extraordinary",
    ]
] = (
    df[
        [
            "Influence",
            "Modesty",
            "Daring",
            "Confidence",
            "Ruling",
            "Responsibility",
            "IntrovExtro",
            "Success",
            "Humble",
            "Leadership",
            "Assertive",
            "Authority",
            "Manipulate",
            "Respect",
            "ShowOffBody",
            "AnalyzePeople",
            "Decision Responsibility",
            "HappyvMeaning",
            "BodyConfidence",
            "ShowOff",
            "KnowWhattoDo",
            "Dependence",
            "Storytelling",
            "Expectations",
            "Satisfaction",
            "Compliments",
            "Power",
            "Trendy",
            "Mirror",
            "CenterofAttention",
            "Freedom",
            "Authority",
            "LeaderPreference",
            "Confidence in Success",
            "MakeBelieve",
            "Leadership Origin",
            "Biography",
            "Public Opinion",
            "Capability",
            "Extraordinary",
        ]
    ].values
    - 1
)

<IPython.core.display.Javascript object>

In [5]:
df

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,1,1,1,1,0,1,0,1,1,...,0,0,1,1,1,0,1,211,1,50
1,6,1,1,1,0,1,1,0,1,0,...,1,0,1,1,1,1,0,149,1,40
2,27,0,1,1,0,1,0,1,0,1,...,0,1,0,0,1,0,1,168,1,28
3,29,0,0,1,1,1,0,1,0,0,...,0,1,0,1,1,0,0,230,1,37
4,6,0,1,0,0,0,1,0,1,0,...,1,0,1,1,1,-1,0,389,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,1,1,1,0,0,1,0,1,0,...,1,0,1,1,1,1,0,243,1,24
11239,10,1,1,0,0,0,1,0,0,0,...,1,0,1,0,1,1,0,197,1,39
11240,6,0,1,1,0,0,1,0,1,0,...,1,0,1,1,1,1,0,447,2,33
11241,12,1,1,0,0,0,0,0,0,0,...,0,1,1,1,1,1,0,167,1,24


<IPython.core.display.Javascript object>

In [9]:
# Remove rows with -1?

<IPython.core.display.Javascript object>

### LASSO Regression

In [11]:
y = df["score"]
X = df.drop(["score"], axis=1)

<IPython.core.display.Javascript object>